In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import profit

In [ ]:
def f(t, x):
    return np.sin(5*x[0] + x[1]*t)

t = np.linspace(0, 8*np.pi, 100)
from profit.util import halton

x = halton.halton(128, 2)
y = []
for xi in x:
    y.append(f(t, xi))
y = np.array(y)
plt.figure()
plt.plot(t, y.T)
plt.show()

In [ ]:
from profit.sur import linear_reduction

kl = linear_reduction.KarhunenLoeve(y, 1e-2)
yrec = kl.lift(kl.project(y))
plt.figure()
plt.plot((yrec-y).T)
plt.show()

In [ ]:
pca = linear_reduction.PCA(y, 1e-2)
yrec2 = pca.lift(pca.project(y))
plt.plot((yrec2-y).T)
plt.show()

In [ ]:
z_kl = kl.project(y)
z_pca = pca.project(y)
print(z_kl.shape)

In [ ]:
import plotly.express as px

zplot = z_kl[-2,:]

px.scatter_3d(x=x[:,0], y=x[:,1], z=zplot, color=zplot)

In [ ]:
px.scatter_3d(x=x[:,0], y=x[:,1], z=z_pca[:,-1], color=z_pca[:,-1])

In [ ]:
from profit.sur import GPySurrogate

k = -1
sm = GPySurrogate()
sm.train(x, z_kl[k,:])
sm.plot()

In [ ]:
# Construct surrogate model for each weight
sms = []
for k in range(len(kl.w)):
    sms.append(GPySurrogate())
    sms[-1].train(x, z_kl[k,:])

In [ ]:
xtest = np.mgrid[0:1:11j,0:1:10j].reshape(2,-1).T

In [ ]:
# Predict weights from surrogate
pred = []
for k in range(len(kl.w)):
    pred.append(sms[k].predict(xtest))
pred = np.array(pred)
print(pred.shape)
zpred = pred[:,0,:,0]
sig2pred = pred[:,1,:,0]
sig2y = kl.Q

In [ ]:
# Decode weights to original space
ktest = 9
V = kl.dy.T @ kl.Q
sig2y = V @ np.diag(sig2pred[:, ktest]) @ V.T + np.cov((yrec-y).T)
confi = 2.0*np.sqrt(np.diag(sig2y))

ypred = kl.lift(zpred)
ytest = np.array([f(t, xi) for xi in xtest])
plt.plot(t, ypred[ktest,:])
plt.fill_between(t, ypred[ktest,:] - confi, ypred[ktest,:] + confi, alpha=0.3)
plt.plot(t, ytest[ktest,:].T)
plt.xlabel('t')
plt.ylabel('f(t;x)')
plt.legend(['Prediction', 'Reference'])
plt.show()

In [ ]:
np.cov((yrec-y).T)

In [ ]:
V @ np.diag(sig2pred[:, ktest]) @ V.T